## Efficient Solutions to Two-Party and Multiparty Millionaires’ Problem

See if we can implement the 2-party ZK comparison protocol outlined in [this paper](https://www.hindawi.com/journals/scn/2017/5207386/)
and perhaps explain what it's doing in simple terms.

### First we need a Paillier cryptosystem:
[Here](./BasicPaillier.ipynb) is a notebook containing one.

In [1]:
%run ./BasicPaillier.ipynb

BacisPaillier notebook loaded


In [2]:
# Note: "Crypto" here is pycryptodome
from Crypto import Random
from Crypto.Random import random
from Crypto.Util import number
import math

## Code for the Millionaires' Problem paper

In [3]:

def Vectorize(U, agb, x):
    '''
    Section 2.5: vectorize the value k, which is a member of public vector U, into a new vector V
    This follows notation in (eq 13)
    ''' 
    (alpha, gamma, beta) = agb
    return [ alpha if u < x else beta if u > x else gamma  for u in U ]


In [4]:


def PiecesDict(names, valBits):
    '''
    Create a dictionary of (pieceName, value) pairs where values are in the same rank-order as 
    the named pieces' ranks
    '''
    vals = [ getRandomBitsOfLength(valBits) for i in range(len(names)) ]
    vals.sort()
    d = { n: v for (n,v) in zip(names,vals) }
    return d
          

In [5]:
# Setup

# stratego pieces in increasing rank
PieceNames = ["Spy","Scout","Miner","Sergeant","Lieutenant","Captain","Major","Colonel","General","Marshal"]; 

CryptoKeyBits = 192

DataBits = 16
PieceVals = PiecesDict(PieceNames, DataBits)

ValNames = { v: n for (n,v) in PieceVals.items() }  # inverted PieceVals dict to look up piece from value

U = list(PieceVals.values())
#random.shuffle(U)

# It doesn't matter whether or not U is shuffled

In [6]:
# section 3.1.1 The Protocol

# Alice's val is x,  Bob's val  is y
A_x = U[random.randint(0,len(U)-1)]

B_y = U[random.randint(0,len(U)-1)]

# The "A_" is to denote that Alice knows this. If she were to send it to Bob, we would create 
# B_x (equal to A_x ) just to denote that Bob knows it. Lame? Sure.

print( "Alice: {}, Bob: {}".format(ValNames[A_x], ValNames[B_y]))

Alice: Marshal, Bob: Major


In [7]:
# step 1  - Alice generates Paillier keys
(A_pubkey, A_privkey) = GeneratePaillierKeys(CryptoKeyBits)

# send pubkey to Bob
B_pubkey = A_pubkey

In [8]:
# step 2  Alice vectorizes x

# There is no guidance for selecting these values, other than that they must all be different.
# Question: since they are going to be encrypted, shouldn't they be random and "look like" ciphers?
A_alpha = 1
A_gamma = 2
A_beta = 3

A_X = Vectorize(U, (A_alpha,A_gamma,A_beta), A_x)

# A_X is really just a list of how x compares to each member of U  
# ie. ["greater", "greater, "less", "equal", "less"... ]

In [9]:
# step 3 - Alice encrypts vector

R = [int(getRandomBitsOfLength(CryptoKeyBits)) for i in range(len(A_X)) ]

A_E = [ int(A_pubkey.encrypt(x,r)) for (x,r) in zip (A_X,R) ]

# A_E is just A_X encrypted so no one but Alice can know how U values compare to x

In [10]:
# step 4  Alice sends E to bob...
B_E = A_E


In [11]:
# step 5 - Bob finds the encrypted version of y in Alice's vector

rb = int(getRandomBitsOfLength(CryptoKeyBits))

# in the paper it says: "He chooses 𝐸(𝑚i,ri)from 𝐸(𝑋) such that 𝑖 = 𝑦 ..."
# which makes no sense: i can never equal y since one is a data value and the 
# other is a small ( < s ) index.  

# I assume they meant "He chooses 𝐸(𝑚i,ri) from 𝐸(𝑋) such that Ui = y..."  
# and it seems to work

# If alice and Bob have U in the same order then Bob probably already knows the index. 
i_for_y = U.index(B_y) # or he can just search for y in U

Ey = B_E[i_for_y]

Ezero = int(B_pubkey.encrypt(0, rb)) # this is a Paillier identity mask. 

# Pailler encryptiom is partially homomorphic, in that
# D( E(y) * E(x) ) = D( E(y + x) )
# so:
# D( E(y) * E(0) ) = D( E(y+0) ) = D( E(y))
# But because of the random r's used in encryption, E(y) is not euqal to E(y+0)
# They just decrypt to the same thing.

B_Eu = Ey * Ezero


In [12]:
#B_Eu

In [13]:
# Step 6  Bob sends his encrypted (and then masked so Alice can't recognize where in her
# vector it came from) back to Alice
A_Eu = B_Eu

In [14]:
# Step 7 - Since Eu was masked by an identity cipher Alice can still decrypt it with her privkey.

u = A_privkey.decrypt(A_Eu)

result = "Alice greater" if u == A_alpha else "Bob greater" if u == A_beta else "Same"

# So the idea here is that Bob has been able to select the Alice-encrypted version of 
# her "x compared to every possible y" list, and then without knowing what it IS, re-mask it 
# into a cipher that still decrypts to the same thing, and send it back to Alice without her
# being able to know WHICH of her comparisons he selected.

result

'Alice greater'